<a href="https://colab.research.google.com/github/park1NG/MLTeamProject/blob/main/acc_per_level_wo_greedsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb

from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')






Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
def data_preprocessing(df):
    df.dropna()
    
    df = df.drop(df[df['예상'] == ':'].index)
    df = df.drop(df[df['도착'] == ':'].index)
    df = df.drop(df[df['현황'] == '취소'].index)

    df['연월일'] = df['연월일'].astype(str)
    df['년'] = df['연월일'].str[:4].astype(int)
    df['월'] = df['연월일'].str[5:7].astype(int)
    df['일'] = df['연월일'].str[8:10].astype(int)

    df['예상'] = df['예상'].astype(str)
    df['예상_1'] = df['예상'].str[:2].astype(int)
    df['예상_2'] = df['예상'].str[3:].astype(int)

    df['도착'] = df['도착'].astype(str)
    df['도착_1'] = df['도착'].str[:2].astype(int)
    df['도착_2'] = df['도착'].str[3:].astype(int)

    df = df[['년', '월', '일', '항공사', '편명', '출발지', '예상_1', '예상_2', '도착_1', '도착_2', '현황']]
    # for col in list(set(df.columns) - set(categorical)):
    #     df = df.astypes({col:'float64'})
    
    categorical = ['항공사', '편명', '출발지', '현황']
    label_encoder = LabelEncoder()
    for col in categorical:
        df[col] = label_encoder.fit_transform(df[col])
        df = df.astype({col:'int'})

    df.info()

    return df


In [1]:
def calculate_f1_score(y_true, y_pred):
    # 정밀도, 재현율, F1 점수 계산
    # micro, macro, samples, weighted
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    return f1

## 0번째 데이터 (file name: train_airportal.csv, test_airportal.csv) 

## 년, 월, 일, 항공사, 편명, 출발지, 예상_1, 예상_2, 도착_1, 도착_2, 현황

In [ ]:
data_train = pd.read_csv('./train_airportal.csv')
data_train = data_preprocessing(data_train)

data_test = pd.read_csv('./test_airportal.csv')
data_test = data_preprocessing(data_test)


x_train_tmp = data_train.iloc[:, :10]  # 0번부터 7번 열까지를 X로 설정
y_train_tmp = data_train.iloc[:, 10]   # 8번 열을 y로 설정

x_test_tmp = data_test.iloc[:, :10]  # 0번부터 7번 열까지를 X로 설정
y_test_tmp = data_test.iloc[:, 10]   # 8번 열을 y로 설정

print(x_test_tmp.info())
print(y_test_tmp.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487397 entries, 0 to 490035
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487397 non-null  int64
 1   월       487397 non-null  int64
 2   일       487397 non-null  int64
 3   항공사     487397 non-null  int64
 4   편명      487397 non-null  int64
 5   출발지     487397 non-null  int64
 6   예상_1    487397 non-null  int64
 7   예상_2    487397 non-null  int64
 8   도착_1    487397 non-null  int64
 9   도착_2    487397 non-null  int64
 10  현황      487397 non-null  int64
dtypes: int64(11)
memory usage: 44.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 182019 entries, 0 to 184516
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       182019 non-null  int64
 1   월       182019 non-null  int64
 2   일       182019 non-null  int64
 3   항공사     182019 non-null  int64
 4   편명      182019 non-null  int64
 5   출발지     18201

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tmp, y_train_tmp)

y_pred = model.predict(x_test_tmp)
accuracy = accuracy_score(y_test_tmp, y_pred)

print("정확도:", accuracy)

정확도: 0.8218867261110104


In [ ]:
print(calculate_f1_score(y_test_tmp, y_pred))

0.8588758767310122


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tmp, y_train_tmp)

y_pred = model.predict(x_test_tmp)
accuracy = accuracy_score(y_test_tmp, y_pred)

print("정확도:", accuracy)

정확도: 0.9342705981243716


In [ ]:
print(calculate_f1_score(y_test_tmp, y_pred))

0.9265630879197277


## 1번째 데이터 (file name: airportal_passenger_airplane_per_day) 

## 예상_승객, 운행_편수 추가(도착 → 실제 출발시간)

In [5]:
per_day_data = pd.read_csv('./per_day.csv')

In [ ]:
per_day_data.info()
per_day_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487396 entries, 0 to 487395
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Unnamed: 0  487396 non-null  int64
 1   년           487396 non-null  int64
 2   월           487396 non-null  int64
 3   일           487396 non-null  int64
 4   항공사         487396 non-null  int64
 5   편명          487396 non-null  int64
 6   출발지         487396 non-null  int64
 7   예상_1        487396 non-null  int64
 8   예상_2        487396 non-null  int64
 9   도착_1        487396 non-null  int64
 10  도착_2        487396 non-null  int64
 11  현황          487396 non-null  int64
 12  예상_승객       487396 non-null  int64
 13  운행_편수       487396 non-null  int64
dtypes: int64(14)
memory usage: 52.1 MB


,Unnamed: 0,년,월,일,항공사,편명,출발지,예상_1,예상_2,도착_1,도착_2,현황,예상_승객,운행_편수
0,0,2016,1,1,49,441,9,0,5,0,11,1,1367,402
1,1,2016,1,1,70,1338,50,0,5,0,30,1,1367,402
2,2,2016,1,1,30,1440,184,0,10,0,15,1,1367,402
3,3,2016,1,1,75,1472,96,0,5,0,55,1,1367,402
4,4,2016,1,1,2,856,4,0,55,1,35,1,1367,402


In [6]:
x_tmp_1, x_tmp_2 = per_day_data.iloc[:, 1:11], per_day_data.iloc[:, 12:] # 독립 변수
y = per_day_data.iloc[:, 11] # 종속 변수
x = pd.concat([x_tmp_1, x_tmp_2], axis=1)

x.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487396 entries, 0 to 487395
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487396 non-null  int64
 1   월       487396 non-null  int64
 2   일       487396 non-null  int64
 3   항공사     487396 non-null  int64
 4   편명      487396 non-null  int64
 5   출발지     487396 non-null  int64
 6   예상_1    487396 non-null  int64
 7   예상_2    487396 non-null  int64
 8   도착_1    487396 non-null  int64
 9   도착_2    487396 non-null  int64
 10  예상_승객   487396 non-null  int64
 11  운행_편수   487396 non-null  int64
dtypes: int64(12)
memory usage: 44.6 MB
<class 'pandas.core.series.Series'>
RangeIndex: 487396 entries, 0 to 487395
Series name: 현황
Non-Null Count   Dtype
--------------   -----
487396 non-null  int64
dtypes: int64(1)
memory usage: 3.7 MB


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9522979072630283


In [ ]:
print(calculate_f1_score(y_test, y_pred))

0.9459081278519333


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
x_train.info()
x_test.info()
y_train.info()
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 389916 entries, 113211 to 121958
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       389916 non-null  int64
 1   월       389916 non-null  int64
 2   일       389916 non-null  int64
 3   항공사     389916 non-null  int64
 4   편명      389916 non-null  int64
 5   출발지     389916 non-null  int64
 6   예상_1    389916 non-null  int64
 7   예상_2    389916 non-null  int64
 8   도착_1    389916 non-null  int64
 9   도착_2    389916 non-null  int64
 10  예상_승객   389916 non-null  int64
 11  운행_편수   389916 non-null  int64
dtypes: int64(12)
memory usage: 38.7 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97480 entries, 448720 to 9253
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   년       97480 non-null  int64
 1   월       97480 non-null  int64
 2   일       97480 non-null  int64
 3   항공사     97480 non-null  int64
 4   편명      9748

In [ ]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.933822322527698


In [ ]:
print(calculate_f1_score(y_test, y_pred))

0.920480207661757


## 2번째 데이터 (file name: airportal_passenger_airplane_per_day) 

## 실제 출발 시간 drop 적용

## 가시거리, 온도, 풍속, 평일, 금요일, 주말, 공휴일 추가

In [7]:
weather_add = pd.read_csv('./holidays_with_weather_fin.csv')

In [8]:
weather_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485994 entries, 0 to 485993
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       485994 non-null  int64  
 1   월       485994 non-null  int64  
 2   일       485994 non-null  int64  
 3   항공사     485994 non-null  int64  
 4   편명      485994 non-null  int64  
 5   출발지     485994 non-null  int64  
 6   예상_1    485994 non-null  int64  
 7   예상_2    485994 non-null  int64  
 8   도착_1    485994 non-null  int64  
 9   도착_2    485994 non-null  int64  
 10  현황      485994 non-null  int64  
 11  예상_승객   485994 non-null  int64  
 12  운행_편수   485994 non-null  int64  
 13  Count   485994 non-null  int64  
 14  가시거리    485994 non-null  float64
 15  온도      485994 non-null  float64
 16  풍속      485994 non-null  float64
 17  평일      485994 non-null  int64  
 18  금요일     485994 non-null  int64  
 19  주말      485994 non-null  int64  
 20  공휴일     485994 non-null  int64  
dtypes: float64

In [14]:
x_tmp_1, x_tmp_2 = weather_add.iloc[:, :10], weather_add.iloc[:, 11:] # 독립 변수
y = weather_add.iloc[:, 10] # 종속 변수
x = pd.concat([x_tmp_1, x_tmp_2], axis=1)

x = x.drop(labels=['도착_1', '도착_2'], axis=1)

x.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485994 entries, 0 to 485993
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       485994 non-null  int64  
 1   월       485994 non-null  int64  
 2   일       485994 non-null  int64  
 3   항공사     485994 non-null  int64  
 4   편명      485994 non-null  int64  
 5   출발지     485994 non-null  int64  
 6   예상_1    485994 non-null  int64  
 7   예상_2    485994 non-null  int64  
 8   예상_승객   485994 non-null  int64  
 9   운행_편수   485994 non-null  int64  
 10  Count   485994 non-null  int64  
 11  가시거리    485994 non-null  float64
 12  온도      485994 non-null  float64
 13  풍속      485994 non-null  float64
 14  평일      485994 non-null  int64  
 15  금요일     485994 non-null  int64  
 16  주말      485994 non-null  int64  
 17  공휴일     485994 non-null  int64  
dtypes: float64(3), int64(15)
memory usage: 66.7 MB
<class 'pandas.core.series.Series'>
RangeIndex: 485994 entries, 0 to 485993
S

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
x_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388795 entries, 339661 to 121958
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       388795 non-null  int64  
 1   월       388795 non-null  int64  
 2   일       388795 non-null  int64  
 3   항공사     388795 non-null  int64  
 4   편명      388795 non-null  int64  
 5   출발지     388795 non-null  int64  
 6   예상_1    388795 non-null  int64  
 7   예상_2    388795 non-null  int64  
 8   예상_승객   388795 non-null  int64  
 9   운행_편수   388795 non-null  int64  
 10  Count   388795 non-null  int64  
 11  가시거리    388795 non-null  float64
 12  온도      388795 non-null  float64
 13  풍속      388795 non-null  float64
 14  평일      388795 non-null  int64  
 15  금요일     388795 non-null  int64  
 16  주말      388795 non-null  int64  
 17  공휴일     388795 non-null  int64  
dtypes: float64(3), int64(15)
memory usage: 56.4 MB
<class 'pandas.core.series.Series'>
Int64Index: 388795 entries, 339661 t

In [17]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9277564583997778


In [18]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9225403553534501


In [19]:
print(calculate_f1_score(y_test, y_pred))

0.9013316771294351


In [ ]:
lgb_classifier = lgb.LGBMClassifier()
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [2, 5, 15]
}

# 그리드 서치를 학습 데이터에 적용하고 최적의 하이퍼파라미터를 찾기
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=10, verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print('Best parameters found by grid search:', grid_search.best_params_)

# 최적의 모델로 예측 및 정확도 측정
y_pred = grid_search.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy with best parameters:', accuracy)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Best parameters found by grid search: {'max_depth': 15, 'n_estimators': 200}
Test accuracy with best parameters: 0.9450405868373132
